In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [6]:
!wget https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/phishtank_phishing_raw_reduced.csv
!wget https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/zenodo_benign_raw.csv

--2024-01-17 04:54:11--  https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/phishtank_phishing_raw_reduced.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3725551 (3.6M) [text/plain]
Saving to: ‘phishtank_phishing_raw_reduced.csv.1’

phishtank_phishing_ 100%[===================>]   3.55M  --.-KB/s    in 0.07s   

2024-01-17 04:54:11 (50.8 MB/s) - ‘phishtank_phishing_raw_reduced.csv.1’ saved [3725551/3725551]

--2024-01-17 04:54:11--  https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/zenodo_benign_raw.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

In [7]:
print("Phishtank Online Valid Dataset")
phishing_df = pd.read_csv("phishtank_phishing_raw_reduced.csv")
phishing_df.head()

Phishtank Online Valid Dataset


,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8110028,https://docs.google.com/presentation/d/e/2PACX...,http://www.phishtank.com/phish_detail.php?phis...,2023-04-08T09:46:13+00:00,yes,2023-04-08T09:53:15+00:00,yes,Other
1,8094828,https://cajaitrujilllo.com/index.php,http://www.phishtank.com/phish_detail.php?phis...,2023-03-28T18:41:15+00:00,yes,2023-03-28T19:09:10+00:00,yes,Other
2,8074198,http://yxbho6gf0p.yennefer6664.pl,http://www.phishtank.com/phish_detail.php?phis...,2023-03-14T09:43:02+00:00,yes,2023-03-17T08:45:23+00:00,yes,Allegro
3,8068992,https://bafybeieykrmvpnwd7ifrl4qjnxeaxfbic4vrv...,http://www.phishtank.com/phish_detail.php?phis...,2023-03-10T17:23:43+00:00,yes,2023-03-11T04:35:34+00:00,yes,Other
4,8033955,https://caloundrawoodworkingclub.com.au/wp-adm...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-11T22:17:45+00:00,yes,2023-02-11T22:22:02+00:00,yes,Other


In [8]:
print("Benign Dataset")
whitelist_df = pd.read_csv("zenodo_benign_raw.csv")
whitelist_df.head()

Benign Dataset


,URLs,Label
0,http://www.wmmayhem.com/,Benign
1,http://www.ballymenaunitedyouthacademy.com/,Benign
2,http://www.brusselsgaybars.com/,Benign
3,http://www.sportsbettingtennis.net/,Benign
4,http://www.i29.mobi/,Benign


# Preprocessing





> **Benign**



In [9]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [10]:
# 1.Domain of the URL (Domain)
def getDomain(url):
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [11]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [12]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1
  else:
    at = 0
  return at

In [13]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0
  else:
    length = 1
  return length

In [14]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [15]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [16]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [17]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [18]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [19]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

In [20]:
# !pip install python-whois

In [21]:
# importing required packages for this section
import requests

In [22]:
# 15. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

In [23]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response):
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

In [24]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [25]:
# 18.Checks the number of forwardings (Web_Forwards)
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [26]:
# calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))

  features.append(label)

  return features

In [27]:
whitelist_df.shape

(19400, 2)

> **Non-phishing**



In [36]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(0, 19400):
  url = whitelist_df['URLs'][i]
  legi_features.append(featureExtraction(url,label))

In [37]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,wmmayhem.com,0,0,0,0,0,0,0,0,0
1,ballymenaunitedyouthacademy.com,0,0,0,0,0,0,0,0,0
2,brusselsgaybars.com,0,0,0,0,0,0,0,0,0
3,sportsbettingtennis.net,0,0,0,0,0,0,0,0,0
4,i29.mobi,0,0,0,0,0,0,0,0,0


In [38]:
# Storing the extracted legitimate URLs fatures to csv file
legitimate.to_csv('cleaned_benign.csv', index= False)

> **Phishing**



In [39]:
phishing_df.shape

(19400, 8)

In [40]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(0, 19400):
  url = phishing_df['url'][i]
  phish_features.append(featureExtraction(url,label))

In [41]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,docs.google.com,0,0,1,5,0,0,0,0,1
1,cajaitrujilllo.com,0,0,0,1,0,0,0,0,1
2,yxbho6gf0p.yennefer6664.pl,0,0,0,0,0,0,0,0,1
3,bafybeieykrmvpnwd7ifrl4qjnxeaxfbic4vrv767gz74o...,0,1,1,1,0,0,0,0,1
4,caloundrawoodworkingclub.com.au,0,0,1,5,0,0,0,0,1


In [42]:
# Storing the extracted legitimate URLs fatures to csv file
phishing.to_csv('cleaned_phishing.csv', index= False)



> **Combined both phishing and non-phishing datasets**



In [43]:
#Concatenating the dataframes into one
combined_benign_phishing = pd.concat([legitimate, phishing]).reset_index(drop=True)
combined_benign_phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,wmmayhem.com,0,0,0,0,0,0,0,0,0
1,ballymenaunitedyouthacademy.com,0,0,0,0,0,0,0,0,0
2,brusselsgaybars.com,0,0,0,0,0,0,0,0,0
3,sportsbettingtennis.net,0,0,0,0,0,0,0,0,0
4,i29.mobi,0,0,0,0,0,0,0,0,0


In [44]:
combined_benign_phishing.tail()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
38795,kundendienst.builderallwppro.com,0,0,1,3,0,0,0,0,1
38796,v.ht,0,0,0,1,0,0,0,0,1
38797,up02-02oapsza2.shopalo.co.pl,0,0,1,4,0,0,0,1,1
38798,kuerennkaycoato-co-jp.kuercnokayccata.zxvtuo.top,0,0,1,1,0,0,0,1,1
38799,1845651992185658.cf,0,0,0,0,0,0,0,0,1


In [45]:
combined_benign_phishing.shape

(38800, 10)

In [46]:
# Storing the data in CSV file
combined_benign_phishing.to_csv('combined_benign_phishing.csv', index=False)